In [1]:
import numpy as np
import pandas as pd
import datetime as dt
import dash
import plotly.express as px
import dash_html_components as html
import dash_core_components as dcc
import dash_bootstrap_components as dbc
from dash.dependencies import Input, Output
from jupyter_dash import JupyterDash
import plotly.graph_objects as go
import warnings

In [2]:
Stats = pd.read_csv("Stats.csv", sep=";")
Summary = pd.read_csv("Summary.csv", sep=";")
Teams = pd.read_csv("Teams.csv", sep= ";")
Players = pd.read_csv("PlayerList.csv", sep= ";")

In [4]:
app = JupyterDash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP])
controls = dbc.Card(
    [
        dbc.FormGroup(
            [
                dbc.Label("Gdzie"),
                dcc.Dropdown(id="Where", 
                options=[{'label': "Dom", "value":"Dom"},
                         {'label': "Wyjazd", "value":"Wyjazd"},
                         {'label': "Oba", "value":"Oba"}
                        ],
                value= "Dom"),
                
            ]
        ),
        dbc.FormGroup(
            [
                dbc.Label("Bramki"),
                dcc.Dropdown(id="Score", 
                options=[{'label': "Stracone", "value":"Stracone"},
                         {'label': "Zdobyte", "value":"Zdobyte"},
                         {'label': "Bilans", "value":"Oba"}
                        ],
                value= "Zdobyte"),
                
            ]
        ),
        dbc.FormGroup(
            [
                dbc.Label("Kolejki"),
                dcc.RangeSlider(
                    id='Kolejki',
                    min=1,
                    max=30,
                    step=1,
                    value=[1, 30]
                ),
                html.Div(id='output-container-range-slider'),   
            ]
        ),
    ],
    body = True,
)
app.layout = dbc.Container(
    [
        html.H1("Liczba zdobytych bramek w sezonie"),
        html.Hr(),
        dbc.Row(
            [
                dbc.Col(controls, md=4),
                dbc.Col(dcc.Graph(id='histogram', figure = px.histogram(data_frame=Stats, x = "Zespol1-Nazwa",
                                                   y = "Zespol1-Bramki")), md=8),
            ],
            align="center",
        ),
    ],
    fluid=True,
)

@app.callback(
    Output('output-container-range-slider', 'children'),
    Output(component_id="histogram", component_property="figure"),
    [
        Input('Kolejki', 'value'),
        Input(component_id="Where", component_property="value"),
        Input(component_id="Score", component_property="value"),
    ],
)
def CreateDashboard(Kolejki, Where, Score):
    who1 = ''
    score1 = ''
    #add description
    description = "Kolejki od " + str(Kolejki[0]) + " do " + str(Kolejki[1])
    #narrows rounds
    StatsForDash = Stats[(Stats["Kolejka"] >=Kolejki[0]) & (Stats["Kolejka"] <=Kolejki[1])]
    figure = px.histogram()
    #Wyjazd
    if Where == "Wyjazd":
        #Zdobyte
        if Score == "Zdobyte":
            figure = px.histogram(data_frame=StatsForDash, x = "Zespol2-Nazwa", y = "Zespol2-Bramki").update_xaxes(
                categoryorder='total descending')
        #Stracone
        elif Score == "Stracone":
        #Bilans wyjazd
            figure = px.histogram(data_frame=StatsForDash, x = "Zespol2-Nazwa", y = "Zespol1-Bramki").update_xaxes(
                categoryorder='total descending')
        else:
            Goals1 = StatsForDash[["Zespol2-Nazwa", "Zespol2-Bramki"]].groupby(['Zespol2-Nazwa']).agg('sum').reset_index()
            Goals2 = StatsForDash[["Zespol2-Nazwa", "Zespol1-Bramki"]].groupby(['Zespol2-Nazwa']).agg('sum').reset_index()
            Goals1["Zespol1-Bramki"] = Goals1["Zespol2-Bramki"] - Goals2["Zespol1-Bramki"]
            figure = px.histogram(data_frame=Goals1, x = "Zespol2-Nazwa", y = "Zespol1-Bramki").update_xaxes(
                categoryorder='total descending')
    #Dom
    elif Where == "Dom":
        #Zdobyte
        if Score == "Zdobyte":
            figure = px.histogram(data_frame=StatsForDash, x = "Zespol1-Nazwa", y = "Zespol1-Bramki").update_xaxes(
                categoryorder='total descending')
            #Stracone
        elif Score == "Stracone":
            figure = px.histogram(data_frame=StatsForDash, x = "Zespol1-Nazwa", y = "Zespol2-Bramki").update_xaxes(
                categoryorder='total descending')
        #Bilans Dom
        else:
            Goals1 = StatsForDash[["Zespol1-Nazwa", "Zespol1-Bramki"]].groupby(['Zespol1-Nazwa']).agg('sum').reset_index()
            Goals2 = StatsForDash[["Zespol1-Nazwa", "Zespol2-Bramki"]].groupby(['Zespol1-Nazwa']).agg('sum').reset_index()
            Goals1["Zespol1-Bramki"] = Goals1["Zespol1-Bramki"] - Goals2["Zespol2-Bramki"]
            figure = px.histogram(data_frame=Goals1, x = "Zespol1-Nazwa", y = "Zespol1-Bramki").update_xaxes(
                categoryorder='total descending')
    #Dom + Wyjazd
    else:
        #Zdobyte
        if Score == "Zdobyte":
            Goals1 = StatsForDash[["Zespol1-Nazwa", "Zespol1-Bramki"]].groupby(['Zespol1-Nazwa']).agg('sum').reset_index()
            Goals2 = StatsForDash[["Zespol2-Nazwa", "Zespol2-Bramki"]].groupby(['Zespol2-Nazwa']).agg('sum').reset_index()
            Goals2.columns = ["Zespol1-Nazwa", "Zespol1-Bramki"]
            Goals1 = Goals1.append(Goals2, ignore_index=True)
            Goals1 = Goals1.groupby(['Zespol1-Nazwa']).agg('sum').reset_index()
            figure = px.histogram(data_frame=Goals1, x = "Zespol1-Nazwa", y = "Zespol1-Bramki").update_xaxes(
                categoryorder='total descending')
        #Stracone
        elif Score == "Stracone":
            Goals1 = StatsForDash[["Zespol1-Nazwa", "Zespol2-Bramki"]].groupby(['Zespol1-Nazwa']).agg('sum').reset_index()
            Goals2 = StatsForDash[["Zespol2-Nazwa", "Zespol1-Bramki"]].groupby(['Zespol2-Nazwa']).agg('sum').reset_index()
            Goals2.columns = ["Zespol1-Nazwa", "Zespol2-Bramki"]
            Goals1 = Goals1.append(Goals2, ignore_index=True)
            Goals1 = Goals1.groupby(['Zespol1-Nazwa']).agg('sum').reset_index()
            figure = px.histogram(data_frame=Goals1, x = "Zespol1-Nazwa", y = "Zespol2-Bramki").update_xaxes(
                categoryorder='total descending')
        else:
            #Zdobyte
            GoalsZ = StatsForDash[["Zespol1-Nazwa", "Zespol1-Bramki"]].groupby(['Zespol1-Nazwa']).agg('sum').reset_index()
            Goals2 = StatsForDash[["Zespol2-Nazwa", "Zespol2-Bramki"]].groupby(['Zespol2-Nazwa']).agg('sum').reset_index()
            Goals2.columns = ["Zespol1-Nazwa", "Zespol1-Bramki"]
            GoalsZ = GoalsZ.append(Goals2, ignore_index=True)
            GoalsZ = GoalsZ.groupby(['Zespol1-Nazwa']).agg('sum').reset_index()
            
            #Stracone
            GoalsS = StatsForDash[["Zespol1-Nazwa", "Zespol2-Bramki"]].groupby(['Zespol1-Nazwa']).agg('sum').reset_index()
            Goals2 = StatsForDash[["Zespol2-Nazwa", "Zespol1-Bramki"]].groupby(['Zespol2-Nazwa']).agg('sum').reset_index()
            Goals2.columns = ["Zespol1-Nazwa", "Zespol2-Bramki"]
            GoalsS = GoalsS.append(Goals2, ignore_index=True)
            GoalsS = GoalsS.groupby(['Zespol1-Nazwa']).agg('sum').reset_index()
            
            GoalsZ["Zespol1-Bramki"] =  GoalsZ["Zespol1-Bramki"] - GoalsS["Zespol2-Bramki"]
            
            figure = px.histogram(data_frame=GoalsZ, x = "Zespol1-Nazwa", y = "Zespol1-Bramki").update_xaxes(
                categoryorder='total descending')
    return description, figure

            
app.run_server(port=8052)

In [145]:
app2 = JupyterDash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP])
controls = dbc.Card(
    [
        dbc.FormGroup(
            [
                dbc.Label("Team"),
                dcc.Dropdown(id="Team", 
                options=[{'label':x, "value":x}
                         for x in sorted(Stats["Zespol1-Nazwa"].unique())
                        ],
                value= "Legia Warszawa"),
                
            ]
        ),
        dbc.FormGroup(
            [
                dbc.Label("Kolejki"),
                dcc.RangeSlider(
                    id='Kolejki',
                    min=1,
                    max=30,
                    step=1,
                    value=[1, 30]
                ),
                html.Div(id='output-container-range-slider2'),   
            ]
        ),
    ],
    body = True,
)
app2.layout = dbc.Container(
    [
        html.H1("Statystyki dotyczace klubow"),
        html.Hr(),
        dbc.Row(
            [
                dbc.Col(controls),
            ]
        ),
        html.Hr(),
        dbc.Row(
            [
                dbc.Col( html.H2("Skumulowana liczba bramek"),
                width={"size": 4, "offset": 1}
                       )
               
            ]
        ),
        html.Hr(),
        dbc.Row(
            [
                dbc.Col(dcc.Graph(id="GoalHome",figure = {}), md=6),
                dbc.Col(dcc.Graph(id="GoalAway",figure = {}),md=6),
            ],
            align="center",
        ),
        html.Hr(),
        dbc.Row(
            [
                dbc.Col( html.H2("Posiadanie piłki"),align = "center",
                width={"size": 4, "offset": 1}
                       )
               
            ]
        ),
        html.Hr(),
        dbc.Row(
            [
                dbc.Col(dcc.Graph(id="Possesion",figure = {})),
            ],
            align="center",
        ),
        html.Hr(),
        dbc.Row(
            [
                dbc.Col( html.H2("Strzały i strzały celne"),
                width={"size": 4, "offset": 1}
                       )
               
            ]
        ),
        html.Hr(),
        dbc.Row(
            [
                dbc.Col(dcc.Graph(id="ShotsHome",figure = {}), md=6),
                dbc.Col(dcc.Graph(id="ShotsAway",figure = {}),md=6),
            ],
            align="center",
        ),
        dbc.Row(
            [
                dbc.Col(html.H3(id="ShotsStatsHome"), width={"size": 6, "offset":1}),
                dbc.Col(html.H3(id="ShotsStatsAway"),width={"size": 4, "offset": 0}),
            ],
            align="center",
        ),
    ],
    fluid=True,
)
@app2.callback(
    Output('output-container-range-slider2', 'children'),
    Output(component_id="GoalHome", component_property="figure"),
    Output(component_id="GoalAway", component_property="figure"),
    Output(component_id="Possesion", component_property="figure"),
    Output(component_id="ShotsHome", component_property="figure"),
    Output(component_id="ShotsAway", component_property="figure"),
    Output('ShotsStatsHome', 'children'),
    Output('ShotsStatsAway', 'children'),
    [
        Input('Kolejki', 'value'),
        Input('Team', 'value'),
    ],
)
def CreateDashboard(Kolejki, Team):
    #add description
    description = "Kolejki od " + str(Kolejki[0]) + " do " + str(Kolejki[1])
    
    #narrows rounds
    StatsForDash = Stats[(Stats["Kolejka"] >=Kolejki[0]) & (Stats["Kolejka"] <=Kolejki[1])]
    
    clubScore = StatsForDash
    clubScore = clubScore.sort_values(by=['Kolejka'])
    
    #GOALS DATA VISUALIZATION WITH PLOTLY
    clubScoreHome = clubScore[(clubScore["Zespol1-Nazwa"] == Team)]
    clubScoreAway = clubScore[(clubScore["Zespol2-Nazwa"] == Team)]
    #Scored
    warnings.filterwarnings("ignore") 
    clubScoreHome['GoleZdobyteSkumulowane'] = [clubScoreHome["Zespol1-Bramki"][clubScoreHome["Kolejka"]<=i].sum() for i in clubScoreHome["Kolejka"]]
    clubScoreAway['GoleZdobyteSkumulowane'] = [clubScoreAway["Zespol2-Bramki"][clubScoreAway["Kolejka"]<=i].sum() for i in clubScoreAway["Kolejka"]]
    #Lost
    clubScoreHome['GoleStraconeSkumulowane'] = [clubScoreHome["Zespol2-Bramki"][clubScoreHome["Kolejka"]<=i].sum() for i in clubScoreHome["Kolejka"]]
    clubScoreAway['GoleStraconeSkumulowane'] = [clubScoreAway["Zespol1-Bramki"][clubScoreAway["Kolejka"]<=i].sum() for i in clubScoreAway["Kolejka"]]
    
    figHome = go.Figure()
    figHome.add_trace(go.Scatter(x=clubScoreHome["Kolejka"], y=clubScoreHome["GoleZdobyteSkumulowane"],
                        mode='lines+markers',
                        name='Zdobyte'))
    figHome.add_trace(go.Scatter(x=clubScoreHome["Kolejka"], y=clubScoreHome["GoleStraconeSkumulowane"],
                        mode='lines+markers',
                        name='Stracone'))
    figHome.update_layout(
        title="W meczach domowych",
        xaxis_title="Kolejki",
        yaxis_title="Liczba bramek",
        legend_title="Bramki",
        font=dict(
            family="Courier New, monospace",
            size=18,
            color="RebeccaPurple"
        )
    )
    figAway = go.Figure()
    figAway.add_trace(go.Scatter(x=clubScoreAway["Kolejka"], y=clubScoreAway["GoleZdobyteSkumulowane"],
                        mode='lines+markers',
                        name='Zdobyte'))
    figAway.add_trace(go.Scatter(x=clubScoreAway["Kolejka"], y=clubScoreAway["GoleStraconeSkumulowane"],
                        mode='lines+markers',
                        name='Stracone'))
    figAway.update_layout(
        title="W meczach wyjazdowych",
        xaxis_title="Kolejki",
        yaxis_title="Liczba bramek",
        legend_title="Bramki",
        font=dict(
            family="Courier New, monospace",
            size=18,
            color="RebeccaPurple"
        )
    )
    
    #POSSESION DATA VISUALIZATION WITH PLOTLY
    
    clubPossesion = StatsForDash
    clubPossesion = clubPossesion.sort_values(by=['Kolejka'])
    
    clubPossesionHome = clubPossesion[(clubPossesion["Zespol1-Nazwa"] == Team)]
    clubPossesionAway = clubPossesion[(clubPossesion["Zespol2-Nazwa"] == Team)]
    
    PossesionHome = clubPossesionHome["Zespol1-Posiadanie piłki"].agg('mean')
    PossesionAway = clubPossesionAway["Zespol2-Posiadanie piłki"].agg('mean')
    PossesionAll = (PossesionAway + PossesionHome)/2
    
    figPossesion = go.Figure()
    figPossesion.add_trace(go.Bar(
        y=['Wyjazd', 'Dom', "Ogolna"],
        x=[PossesionAway, PossesionHome, PossesionAll],
        name=Team,
        orientation='h',
        marker=dict(
            color='rgba(246, 78, 139, 0.6)',
            line=dict(color='rgba(246, 78, 139, 1.0)', width=3)
        )
    ))
    figPossesion.add_trace(go.Bar(
        y=['Wyjazd', 'Dom', "Ogolna"],
        x=[100-PossesionAway, 100-PossesionHome, 100-PossesionAll],
        name='Rywale',
        orientation='h',
         marker=dict(
            color='rgba(58, 71, 80, 0.6)',
            line=dict(color='rgba(58, 71, 80, 1.0)', width=3)
        )
    ))
    figPossesion.update_layout(barmode='stack',
                               font=dict(
                                   family="Courier New, monospace",
                                   size=18,
                                   color="RebeccaPurple"
            ))
    
    #Shots DATA VISUALIZATION WITH PLOTLY
    clubShotsHome = clubScore[(clubScore["Zespol1-Nazwa"] == Team)]
    clubShotsAway = clubScore[(clubScore["Zespol2-Nazwa"] == Team)]
    figHomeShots = go.Figure()
    figHomeShots.add_trace(go.Scatter(x=clubShotsHome["Kolejka"], y=clubShotsHome["Zespol1-Strzały"],
                        mode='lines+markers',
                        name='W sumie'))
    figHomeShots.add_trace(go.Scatter(x=clubShotsHome["Kolejka"], y=clubShotsHome["Zespol1-Strzały celne"],
                        mode='lines+markers',
                        name='Celne'))
    figHomeShots.update_layout(
        title="W meczach domowych",
        xaxis_title="Kolejki",
        yaxis_title="Liczba strzałów",
        legend_title="Strzały",
        font=dict(
            family="Courier New, monospace",
            size=18,
            color="RebeccaPurple"
        )
    )
    figAwayShots = go.Figure()
    figAwayShots.add_trace(go.Scatter(x=clubShotsAway["Kolejka"], y=clubShotsAway["Zespol2-Strzały"],
                        mode='lines+markers',
                        name='W sumie'))
    figAwayShots.add_trace(go.Scatter(x=clubShotsAway["Kolejka"], y=clubShotsAway["Zespol2-Strzały celne"],
                        mode='lines+markers',
                        name='Celne'))
    figAwayShots.update_layout(
        title="W meczach wyjazdowych",
        xaxis_title="Kolejki",
        yaxis_title="Liczba strzałów",
        legend_title="Strzały",
        font=dict(
            family="Courier New, monospace",
            size=18,
            color="RebeccaPurple"
        )
    )
    
    homeS = clubShotsHome['Zespol1-Strzały'].sum()
    awayS = clubShotsAway['Zespol2-Strzały'].sum()
    shotsHome = 0
    shotsAway = 0
    
    if homeS != 0:
        shotsHome = clubShotsHome['Zespol1-Strzały celne'].sum() / homeS
    if awayS != 0:
        shotsAway = clubShotsAway['Zespol2-Strzały celne'].sum() / awayS
    
    shotsHome = str(round(shotsHome, 2)) + "% celności strzałów"
    shotsAway = str(round(shotsAway, 2)) + "% celności strzałów"
    return description, figHome, figAway, figPossesion, figHomeShots, figAwayShots, shotsHome, shotsAway

app2.run_server(port=8051)

Dash app running on http://127.0.0.1:8051/
